In [32]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [28]:
  # Function to create input sequences for LSTM model
def create_sequences(df, features, target, n_steps=10):
    try:
        X, y = [], []
        data = df[features].values
        for i in range(len(data) - n_steps):
            X.append(data[i:i + n_steps]) # Input sequence
            y.append(data[i + n_steps][features.index(target)]) # Target value
        X, y = np.array(X), np.array(y)
        print("Sequences created successfully.")
        return X, y
    except Exception as e:
        print(f"Error in creating sequences: {e}")
        return None, None

In [29]:
# Function to build the LSTM model
def build_lstm_model(input_shape):
    try:
        model = Sequential()
        model.add(LSTM(units=50, activation='relu', input_shape=input_shape))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
        print("LSTM model built successfully.")
        return model
    except Exception as e:
        print(f"Error in building LSTM model: {e}")
        return None

In [30]:
# Function to train the LSTM model
def train_model(model, X_train, y_train, X_test, y_test):
    try:
        # Ensure the data is in the correct format
        X_train = np.array(X_train, dtype=np.float32)
        y_train = np.array(y_train, dtype=np.float32)
        X_test = np.array(X_test, dtype=np.float32)
        y_test = np.array(y_test, dtype=np.float32)

        # Setup early stopping to prevent overfitting
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        # Train the model
        model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=1, callbacks=[early_stopping])
        print("Model trained successfully.")
    except Exception as e:
        print(f"Error in training model: {e}")

In [33]:
preprocessed_stock_data = pd.read_csv('/content/preprocessed_stock_data.csv') # Read the preprocessed dataset
preprocessed_stock_data

# Define the features
features = preprocessed_stock_data.columns.tolist()

# Define the target variable
target = 'last_value'

# Create input sequences for LSTM model
X, y = create_sequences(preprocessed_stock_data, features, target)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

model = build_lstm_model((X_train.shape[1], X_train.shape[2]))
if model:
    train_model(model, X_train, y_train, X_test, y_test)

Sequences created successfully.
LSTM model built successfully.
Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0724 - val_loss: 0.0059
Epoch 2/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0028 - val_loss: 0.0035
Epoch 3/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0011 - val_loss: 0.0034
Epoch 4/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 7.4579e-04 - val_loss: 0.0036
Epoch 5/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 6.0388e-04 - val_loss: 0.0033
Epoch 6/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 5.5023e-04 - val_loss: 0.0025
Epoch 7/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4.1722e-04 - val_loss: 0.0028
Epoch 8/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 3.7498e-04 - val_loss: 0.0030
Epoch 9/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 4.3876e-04 - val_loss: 0.0020
Epoch 10/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 3.1858e-04 - val_loss: 0.0023
Epoch 11/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 3.4290e-04 - val_loss: 0.0018
Epoch 12/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s

In [35]:
# Save the trained model
model.save('lstm_stock_price_model.keras')
print("Model saved to 'lstm_stock_price_model.keras'")

Model saved to 'lstm_stock_price_model.keras'
